In [1]:
import ROOT as rt
import csv
import re
import sys
import collections
from collections import OrderedDict
import uproot
import numpy as np
import time
import numba
from numba import jit
from matplotlib import pyplot as plt
from numpy import linalg as LA
from ROOT import TLorentzVector
from array import array

import math

# sys.path.append('/home/cms/Desktop/LLP/cms_lpc_llp/jet_timing_studies/python/lib')
# sys.path.append('/nfshome/christiw/llp/cms_lpc_llp/jet_timing_studies/python/lib')
sys.path.append('/nfshome/christiw/llp/delayed_jet_analyzer/lib/')
# sys.path.append('/nfshome/christiw/llp/cms_lpc_llp/llp_analyzer/python/lib')

from histo_utilities import create_TH1D, create_TH2D, create_TGraph,std_color_list
     
import CMS_lumi, tdrstyle 
tdrstyle.setTDRStyle()
CMS_lumi.writeExtraText = 1
CMS_lumi.extraText = "     Simulation Preliminary"

# donotdelete = []
print(sys.version)

Welcome to JupyROOT 6.16/00
3.6.6 (default, Mar 29 2019, 00:03:27) 
[GCC 4.8.5 20150623 (Red Hat 4.8.5-36)]


In [2]:
fpath_bkg =OrderedDict()
tree_bkg = OrderedDict()
tree_sig = OrderedDict()
fpath_sig =OrderedDict()

bkg_path = "/nfshome/christiw/llp/jet_timing_studies_root/trackless/"
lumi = 137000 #in pb-1
fpath_bkg['WJetsToLNu'] = bkg_path+"jet_timing_studies_ntuple_RunIIFall17DRPremix_WJetsToLNu_TuneCP5_13TeV-madgraphMLM-pythia8.root"

fpath_bkg['QCD'] = bkg_path+"jet_timing_studies_ntuple_RunIIFall17DRPremix_QCD_Pt_170to300_TuneCP5_13TeV_pythia8.root"
fpath_bkg['bbbb_mh125_ctau1'] = bkg_path+'jet_timing_studies_ntuple_bbbb_vh_ISR_mh125_mx50_pl1000_ev100000.root'

xsec = {}
xsec['QCD'] = 117989*0.07335
xsec['WJetsToLNu'] = 61526.7
xsec['bbbb_mh125_ctau1'] = 1.475E+00

weight = {}


for k,v in fpath_bkg.items():
    print (k, v)
#     tree[k] = rtnp.root2array(v)
    root_dir = uproot.open(v) 
    
    NEvents = root_dir['ntuples']['NEvents'][1]
    print("Number of Events: "+str(NEvents))
    weight[k] = xsec[k]*lumi/NEvents
    print("weight is: "+str(weight[k]))
    tree_bkg[k] = root_dir['ntuples']['llp']
    a = tree_bkg[k]["jetPt"].array()
    print (a[:-1][0])
    print(a.shape)

WJetsToLNu /nfshome/christiw/llp/jet_timing_studies_root/trackless/jet_timing_studies_ntuple_RunIIFall17DRPremix_WJetsToLNu_TuneCP5_13TeV-madgraphMLM-pythia8.root
Number of Events: 10000.0
weight is: 842915.79
[24.78586]
(10000,)
QCD /nfshome/christiw/llp/jet_timing_studies_root/trackless/jet_timing_studies_ntuple_RunIIFall17DRPremix_QCD_Pt_170to300_TuneCP5_13TeV_pythia8.root
Number of Events: 59068.0
weight is: 20072.891608823727
[231.354     87.641464  56.886036]
(59068,)
bbbb_mh125_ctau1 /nfshome/christiw/llp/jet_timing_studies_root/trackless/jet_timing_studies_ntuple_bbbb_vh_ISR_mh125_mx50_pl1000_ev100000.root
Number of Events: 74588.0
weight is: 2.709215959671797
[64.42451  51.139175 37.67556 ]
(74588,)


In [3]:
trigger_names_file = '/home/cms/Desktop/LLP/delayed_jet_analyzer/data/trigger_names_llp_v1.dat'
trigger_names_file = '/nfshome/christiw/llp/cms_lpc_llp/jet_timing_studies/data/trigger_names_llp_v1.dat'

trigger_names = []
with open(trigger_names_file) as f:
    reader = csv.reader(f, delimiter=" ")
    for line in reader:
        trigger_names.append(line[2])
    trigger_paths = [87,135] #PFMET120


# 177 HLT_Mu17_TrkIsoVVL_Mu8_TrkIsoVVL_DZ
# 362 HLT_Ele23_Ele12_CaloIdL_TrackIdL_IsoVL_DZ
# 310 HLT_PFMET120_PFMHT120_IDTight
# 87 HLT_Ele32_WPTight_Gsf
# 135 HLT_IsoMu24
# 310 PFMET

In [4]:
jet_branches = {}
calojet_branches = {}
event_branches = {}

In [5]:
def frange(start,end,step):
    result = [start]
    counter = start
    while abs(counter-end)>=step*0.9:
        counter+=step
        result.append(counter)
    return result



In [11]:
ptcut = 20
nTags = {}
calojetGammaMax = {}
calojetMinDeltaRPVTracks = {}
calojetPt = {}
nCaloJets = {}
calojetRechitT = {}
# mindeltaR_cuts = frange(0.05,0.15,0.01)
# time_cuts = frange(0.5,2.5,0.1)
# gamma_cuts = frange(0.05,0.25,0.01)
mindeltaR_cuts =[0.05]
time_cuts = [0.5]
gamma_cuts = [0.17]
for gamma_cut in gamma_cuts:
    for time_cut in time_cuts:
        for mindeltaR_cut in mindeltaR_cuts:
            for k,T in tree_bkg.items():
                nTags[k] = [] #event level
                hlt = T['HLTDecision'].array()
                sel_ev = np.zeros(hlt[:,0].shape)
                for tr in trigger_paths:
                    sel_ev  = np.logical_or(sel_ev,hlt[:,tr])
                calojetPt[k] =T['calojetPt'].array()[sel_ev]
                nCaloJets[k] = T['nCaloJets'].array()[sel_ev]
                calojetGammaMax[k] = T['calojetBetaMax'].array()[sel_ev]
                calojetMinDeltaRPVTracks[k] = T['calojetMinDeltaRPVTracks'].array()[sel_ev]
                calojetRechitT[k] = T['calojetRechitT'].array()[sel_ev]
                for ev in range(len(calojetPt[k])):#loop over each event
                    tag = 0
                    for j in range(nCaloJets[k][ev]):
                        if calojetPt[k][ev][j] < ptcut: continue
                        if calojetGammaMax[k][ev][j]<gamma_cut and calojetMinDeltaRPVTracks[k][ev][j]>mindeltaR_cut and calojetRechitT[k][ev][j]>time_cut:
                            tag += 1
                    nTags[k].append(tag)
                nTags[k] = np.array(nTags[k])

            # Plotting THStack
            c = rt.TCanvas('c','c', 800, 800)
            h = {}
            # leg = rt.TLegend(0.59,0.70,0.97,0.93)
            leg = rt.TLegend(0.59,0.74,0.93,0.91)

            leg.SetTextSize(0.025)
            # leg.SetTextFont(42)
            leg.SetEntrySeparation(0.01)
            stack = rt.THStack( "hs" , "Hgg Stack " )
            # color_list = [625,597,876,806,861,393,845]
            color_list = [845,800,861,806,876,597,625]
            i = 0
            max_y = 0
            for k,v in nTags.items():
                if k == 'WJetsToLNu' or k == 'QCD':
                    w = list(weight[k]*np.ones(v.shape))
                    h[k] = create_TH1D( v, axis_title=['nTags', 'Events'], name=k, binning=[5,0,5],weights =w)
                    stack.Add( h[k])
                    leg.AddEntry(h[k],k)
                    h[k].SetLineColor(color_list[i])
                    h[k].SetFillColor(color_list[i])
                    i+=1

                    #find max y to set y axis
                    currentMax=h[k].GetBinContent(h[k].GetMaximumBin())

                    if currentMax > max_y:
                        max_y = currentMax
                        h[k].SetMaximum(10*currentMax)
                        h[k].SetMinimum(1)

            i = 0
            stack.Draw('HIST')
            for k,v in nTags.items():
                if k[:4] == 'bbbb':
                    h[k] = create_TH1D( v, axis_title=['nTags', 'Events'], name=k, binning=[5,0,5],weights = weight[k]*np.ones(v.shape))
                    h[k].SetLineColor(std_color_list[i])


                    leg.AddEntry(h[k],k)

                h[k].Draw('samehist')
                i+=1

            # print s/sqrt(s+b)
            purity = 0
            for i in range(5):
                last = stack.GetStack().Last()
                b = last.GetBinContent(i+1)
                s = h['bbbb_mh125_ctau1'].GetBinContent(i+1)
                if not s == 0 and b == 0:
                    purity += (s/(s+b)**0.5)**2
            purity = purity **0.5
            if purity > 22:
                print(gamma_cut,time_cut, mindeltaR_cut, purity)
                
                
                leg.Draw()
                stack.GetXaxis().SetTitle('Number of tagged jets')
                stack.GetYaxis().SetTitle("Events")
                c.SetLeftMargin(0.2)
                stack.GetYaxis().SetTitleOffset(1.6)

                stack.SetMaximum(max_y*10)

                stack.SetMinimum(10)
                CMS_lumi.lumi_13TeV = "137 fb^{-1}"
                CMS_lumi.cmsText = ""
                CMS_lumi.writeExtraText = False
                CMS_lumi.CMS_lumi(c,4,3)


                # if setlog: 
                #     c.SetLogy()
                c.SetLogy()
                c.Draw()
                c.SaveAs("../plots/ntags_optimization/mindeltaRcut"+str(mindeltaR_cut)+"_timecut"+str(time_cut)+"gammacut"+str(gamma_cut)+".pdf")

0.17 0.5 0.05 24.07845957219366
137 fb^{-1} (13 TeV)


Warning in <TCanvas::Constructor>: Deleting canvas with same name: c
Warning in <TROOT::Append>: Replacing existing TH1: WJetsToLNu (Potential memory leak).
Warning in <TROOT::Append>: Replacing existing TH1: QCD (Potential memory leak).
Info in <TCanvas::Print>: pdf file ../plots/ntags_optimization/mindeltaRcut0.05_timecut0.5gammacut0.17.pdf has been created


[92478.72008952571, 92478.72008952571, 92478.72008952571, 92478.72008952571, 92478.72008952571, 92478.72008952571, 92478.72008952571, 92478.72008952571, 92478.72008952571, 92478.72008952571]
[92478.72008952571, 92478.72008952571, 92478.72008952571, 92478.72008952571, 92478.72008952571, 92478.72008952571, 92478.72008952571, 92478.72008952571, 92478.72008952571, 92478.72008952571]
137 fb^{-1} (13 TeV)


Warning in <TCanvas::Constructor>: Deleting canvas with same name: c
Warning in <TROOT::Append>: Replacing existing TH1: WJetsToLNu (Potential memory leak).
Warning in <TROOT::Append>: Replacing existing TH1: QCD (Potential memory leak).
Info in <TCanvas::Print>: pdf file ../plots/test.pdf has been created
libpng warning: Application built with libpng-1.5.13 but running with 1.6.21
Error in <TASImage::WriteImage>: error writing file ../plots/test.png
Info in <TCanvas::Print>: png file ../plots/test.png has been created


In [42]:
# Plotting pf time vs calo time

max_y = 0
j = 0
c = rt.TCanvas('c','c', 800, 800)
h = {}
# leg = rt.TLegend(0.50,0.75,0.97,0.93)
leg = rt.TLegend(0.65,0.80,0.95,0.93)
leg.SetTextSize(0.022)
# leg.SetTextFont(42)
leg.SetEntrySeparation(0.01)
print(jetRechitT[k].shape)
print(calojetRechitT[k].shape)
h['pf'] = create_TH1D(jetRechitT['bbbb_mh125_ctau1'], axis_title = ['t_{reco} [ns]','Events'], binning=[50,-20,20])

h['calo'] = create_TH1D(calojetRechitT['bbbb_mh125_ctau1'], axis_title = ['t_{reco} [ns]','Events'], binning=[50,-20,20])
   
h['pf'].SetLineColor(std_color_list[0])
h['calo'].SetLineColor(std_color_list[1])

leg.AddEntry(h['pf'], 'PF Jet')
leg.AddEntry(h['calo'], 'Calo Jet')

h['pf'].DrawNormalized('')
h['calo'].DrawNormalized('same')

# c.SetRightMargin(0.17)

leg.Draw()


c.SetLogy()
c.Draw()
c.SaveAs('../plots/trackless/treco_pfvscalo.pdf')



(195069,)
(131487,)


Warning in <TCanvas::Constructor>: Deleting canvas with same name: c
Warning in <TROOT::Append>: Replacing existing TH1: h (Potential memory leak).
Info in <TCanvas::Print>: pdf file ../plots/trackless/treco_pfvscalo.pdf has been created


In [41]:
# Plotting mass
var_calo = [calojetAlphaMax,calojetBetaMax,calojetGammaMax_EM,calojetGammaMax_Hadronic,calojetGammaMax, log10calojetMedianIP, calojetMinDeltaRAllTracks,calojetMinDeltaRPVTracks]
axis_calo = [['caloJet #alpha_{Max}','Events'],['caloJet #beta_{Max}','Events'],['caloJet #gamma_{EM Max}','Events'],['caloJet #gamma_{Had Max}','Events'],['caloJet #gamma_{Max}','Events'],['caloJet log10(\hat{IP}_{sig})','Events'],['caloJet all tracks #Delta R_{min}','Events'],['caloJet PV tracks #Delta R_{min}','Events']]
name_calo = ['calojetAlphaMax','calojetBetaMax','calojetGammaMax_EM','calojetGammaMax_Hadronic','calojetGammaMax', 'calojetMedianIP', 'calojetMinDeltaRAllTracks','calojetMinDeltaRPVTracks']
bins_calo = [[50,0,1],[50,0,1],[50,0,1],[50,0,1],[50,0,1],[50,-1,3],[50,0,1],[50,0,1]]

var_pf = [CEF,jetAlphaMax,jetBetaMax,jetGammaMax_EM,jetGammaMax_Hadronic,jetGammaMax, log10jetMedianIP, jetMinDeltaRAllTracks,jetMinDeltaRPVTracks,]
axis_pf = [['CEF','Events'],['PFJet #alpha_{Max}','Events'],['PFJet #beta_{Max}','Events'],['PFJet #gamma_{EM Max}','Events'],['PFJet #gamma_{Had Max}','Events'],['PFJet #gamma_{Max}','Events'],['PFJet log10(\hat{IP}_{sig})','Events'],['PFJet all tracks #Delta R_{min}','Events'],['PFJet PV tracks #Delta R_{min}','Events']]
name_pf = ['CEF','pfjetAlphaMax','pfjetBetaMax','pfjetGammaMax_EM','pfjetGammaMax_Hadronic','pfjetGammaMax', 'pfjetMedianIP', 'pfjetMinDeltaRAllTracks','pfjetMinDeltaRPVTracks',]
bins_pf = [[50,0,1],[50,0,1],[50,0,1],[50,0,1],[50,0,1],[50,0,1],[50,-1,3],[50,0,1],[50,0,1]]


var = var_pf + var_calo
axis = axis_pf+ axis_calo
name = name_pf+name_calo
bins = bins_pf+bins_calo
print(len(var),len(axis),len(name),len(bins))
for i in range(len(var)):
    max_y = 0
    j = 0
    c = rt.TCanvas('c','c', 800, 800)
    h = {}
    # leg = rt.TLegend(0.50,0.75,0.97,0.93)
    leg = rt.TLegend(0.65,0.80,0.95,0.93)
    leg.SetTextSize(0.022)
    # leg.SetTextFont(42)
    leg.SetEntrySeparation(0.01)

    for k,v in tree_bkg.items():
        h[k] = create_TH1D(var[i][k],axis_title = axis[i],binning = bins[i])

        currentMax=h[k].GetBinContent(h[k].GetMaximumBin())

        if currentMax > max_y:
            max_y = currentMax
            h[k].SetMaximum(10*currentMax)
            h[k].SetMinimum(1)
        h[k].GetXaxis().SetLabelSize(0.045)

    #         h[k].GetXaxis().SetTickLength(0.)
        h[k].SetLineColor(std_color_list[j])
        leg.AddEntry(h[k], k)
        h[k].DrawNormalized('same')
        j+=1

# c.SetRightMargin(0.17)

        leg.Draw()


        c.SetLogy()
        c.Draw()
        c.SaveAs('../plots/trackless/'+name[i]+'_signalbkg.pdf')



17 17 17 17


Warning in <TCanvas::Constructor>: Deleting canvas with same name: c
Info in <TCanvas::Print>: pdf file ../plots/trackless/CEF_signalbkg.pdf has been created
Warning in <TROOT::Append>: Replacing existing TH1: h (Potential memory leak).
Info in <TCanvas::Print>: pdf file ../plots/trackless/CEF_signalbkg.pdf has been created
Warning in <TCanvas::Constructor>: Deleting canvas with same name: c
Info in <TCanvas::Print>: pdf file ../plots/trackless/pfjetAlphaMax_signalbkg.pdf has been created
Warning in <TROOT::Append>: Replacing existing TH1: h (Potential memory leak).
Info in <TCanvas::Print>: pdf file ../plots/trackless/pfjetAlphaMax_signalbkg.pdf has been created
Warning in <TCanvas::Constructor>: Deleting canvas with same name: c
Info in <TCanvas::Print>: pdf file ../plots/trackless/pfjetBetaMax_signalbkg.pdf has been created
Warning in <TROOT::Append>: Replacing existing TH1: h (Potential memory leak).
Info in <TCanvas::Print>: pdf file ../plots/trackless/pfjetBetaMax_signalbkg.pdf h

In [44]:
### ROC curve
# var = [CEF,calojetAlphaMax,calojetAlphaMax_jetpt,calojetBetaMax_EM,calojetBetaMax_Hadronic,calojetBetaMax]
# name = ['CEF','calojetAlphaMax','calojetAlphaMax_jetpt','calojetBetaMax_EM','calojetBetaMax_Hadronic','calojetBetaMax']
calo = 1
c = rt.TCanvas('c','c', 800, 800)
leg = rt.TLegend(0.18,0.76,0.53,0.94)
leg.SetTextSize(0.022)
leg.SetEntrySeparation(0.01)
gr = {}
var_calo = [calojetAlphaMax,calojetBetaMax,calojetGammaMax_EM,calojetGammaMax_Hadronic,calojetGammaMax, log10calojetMedianIP, calojetMinDeltaRAllTracks,calojetMinDeltaRPVTracks]
axis_calo = [['caloJet #alpha_{Max}','Events'],['caloJet #beta_{Max}','Events'],['caloJet #gamma_{EM Max}','Events'],['caloJet #gamma_{Had Max}','Events'],['caloJet #gamma_{Max}','Events'],['caloJet log10(\hat{IP}_{sig})','Events'],['caloJet all tracks #Delta R_{min}','Events'],['caloJet PV tracks #Delta R_{min}','Events']]
name_calo = ['calojetAlphaMax','calojetBetaMax','calojetGammaMax_EM','calojetGammaMax_Hadronic','calojetGammaMax', 'calojetMedianIP', 'calojetMinDeltaRAllTracks','calojetMinDeltaRPVTracks']
bins_calo = [[50,0,1],[50,0,1],[50,0,1],[50,0,1],[50,0,1],[50,-1,3],[50,0,1],[50,0,1]]

var_pf = [CEF,jetAlphaMax,jetBetaMax,jetGammaMax_EM,jetGammaMax_Hadronic,jetGammaMax, log10jetMedianIP, jetMinDeltaRAllTracks,jetMinDeltaRPVTracks,]
axis_pf = [['CEF','Events'],['PFJet #alpha_{Max}','Events'],['PFJet #beta_{Max}','Events'],['PFJet #gamma_{EM Max}','Events'],['PFJet #gamma_{Had Max}','Events'],['PFJet #gamma_{Max}','Events'],['PFJet log10(\hat{IP}_{sig})','Events'],['PFJet all tracks #Delta R_{min}','Events'],['PFJet PV tracks #Delta R_{min}','Events']]
name_pf = ['CEF','pfjetAlphaMax','pfjetBetaMax','pfjetGammaMax_EM','pfjetGammaMax_Hadronic','pfjetGammaMax', 'pfjetMedianIP', 'pfjetMinDeltaRAllTracks','pfjetMinDeltaRPVTracks',]
bins_pf = [[50,0,1],[50,0,1],[50,0,1],[50,0,1],[50,0,1],[50,0,1],[50,-1,3],[50,0,1],[50,0,1]]




if calo:
    var = var_calo
    name = name_calo
    axis = axis_calo
else:
    var = var_pf
    name = name_pf
    axis = axis_pf
for i in range(len(var)):
    v = var[i]
    eff = {}
    thre = np.array(range(1000))/100.0-5
    for k in tree_bkg.keys():
        eff[k] = []
        for th in thre:
            if 'IP' in name[i] or 'MinDeltaR' in name[i]:
                eff[k].append(1.0*np.count_nonzero(v[k]>th)/len(v[k]))
            else:
                eff[k].append(1.0*np.count_nonzero(v[k]<th)/len(v[k]))
#     Plotting ROC curve

    gr[name[i]] = create_TGraph(eff['QCD'],eff['bbbb_mh125_ctau1'],axis_title = ['#epsilon_{bkg}','#epsilon_{signal}'])
    gr[name[i]].SetLineColor(std_color_list[i])
    gr[name[i]].SetLineWidth(2)
    gr[name[i]].SetMarkerColor(std_color_list[i])
    leg.AddEntry(gr[name[i]],axis[i][0])

    gr[name[i]].Draw('ac' if i==0 else 'c')
leg.Draw()
c.Draw()
if calo:
    c.SaveAs('../plots/trackless/caloROC.pdf')
else:
    c.SaveAs('../plots/trackless/pfROC.pdf')

/usr/local/lib/python3.6/site-packages/ipykernel_launcher.py:41: RuntimeWarning: invalid value encountered in less
Warning in <TCanvas::Constructor>: Deleting canvas with same name: c
Info in <TCanvas::Print>: pdf file ../plots/trackless/caloROC.pdf has been created
